In [ ]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

print("正在加载句向量模型")
model = SentenceTransformer('./text2vec_local', device='cuda')

# 加载原始训练数据
train_file = 'train_data.jsonl'
print(f"正在读取训练数据从 {train_file}...")
with open(train_file, 'r', encoding='utf-8') as f:
    train_data = [json.loads(line) for line in f if line.strip()]

# 提取需要被索引的内容和对应的答案
user_contents = [entry['messages'][1]['content'] for entry in train_data]
assistant_responses = [entry['messages'][2]['content'] for entry in train_data]

print(f"正在将 {len(user_contents)} 条训练数据编码为向量...")
embeddings = model.encode(user_contents, show_progress_bar=True, normalize_embeddings=True)
embeddings = np.array(embeddings).astype('float32')

# 创建并构建FAISS索引
embedding_dim = embeddings.shape[1]
index = faiss.IndexFlatIP(embedding_dim)
index.add(embeddings)

print(f"正在保存FAISS索引到 'knowledge_base.index'")
faiss.write_index(index, 'knowledge_base.index')

# 同时保存原始内容以便后续查找
knowledge_base_content = {
    'user_inputs': user_contents,
    'assistant_responses': assistant_responses
}
with open('knowledge_base_content.json', 'w', encoding='utf-8') as f:
    json.dump(knowledge_base_content, f, ensure_ascii=False, indent=2)

print("知识库构建完成！")

正在加载句向量模型 (这可能需要一些时间)...
正在读取训练数据从 train_data.jsonl...
正在将 4000 条训练数据编码为向量...


Batches:   0%|          | 0/125 [00:00<?, ?it/s]

正在保存FAISS索引到 'knowledge_base.index'
知识库构建完成！


In [ ]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# ============ 1. 加载知识库和检索模型 ============
print("正在加载检索工具...")
retrieval_model = SentenceTransformer('./text2vec_local', device='cuda')
index = faiss.read_index('knowledge_base.index')
with open('knowledge_base_content.json', 'r', encoding='utf-8') as f:
    knowledge_base = json.load(f)

# ============ 2. 路径和模型配置 (与您原版相同) ============
base_model_path = "./Qwen3-8B"
lora_model_path = "./qwen3-8b-lora-finetuned-2/final_checkpoint"
test_file_path = "test1.json"
output_file_path = "output_srag_augmented.txt" # 使用新文件名以作区分

# 基础的 System Prompt
base_system_prompt = (
    "请从文本中抽取仇恨言论四元组，要求：\n"
    "1. 严格按照以下格式回复：(评论对象 | 论点 | 目标群体 | 是否仇恨 [END])，直接输出，不要解释。\n"
    "2. 如有多个四元组，两两之间用[SEP]分隔。\n"
    "3. 目标群体可以包含以下6项中的一项或多项：Region、Racism、Sexism、LGBTQ、others、non-hate。"
    "注意仅当‘是否仇恨’项为‘non-hate’时，‘目标群体’项才为‘non-hate’。"
)

# ============ 3. 加载LLM============
print("加载LLM中...")
tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)
model = PeftModel.from_pretrained(base_model, lora_model_path)
model.eval()

# ============ 4. 读取测试数据============
with open(test_file_path, "r", encoding="utf-8") as f:
    test_data = json.load(f)

# ============ 5. 批量推理主循环 (集成SRAG) ============
print(f"开始使用SRAG进行推理，结果将写入 {output_file_path}")
with open(output_file_path, "w", encoding="utf-8") as out_file:
    for idx, example in enumerate(test_data):
        user_input = example["content"]

        # --- SRAG核心: 检索与构建Prompt ---
        # a. 将当前输入编码为向量
        query_embedding = retrieval_model.encode([user_input], normalize_embeddings=True).astype('float32')
        
        # b. 在FAISS中搜索最相似的2个例子
        k = 2
        distances, indices = index.search(query_embedding, k)
        
        # c. 构建 few-shot 范例
        few_shot_examples = "下面是一些范例，仅供参考，请学习它们的格式，但要根据新的用户输入来生成内容。：\n---"
        for i in indices[0]:
            retrieved_input = knowledge_base['user_inputs'][i]
            retrieved_output = knowledge_base['assistant_responses'][i]
            few_shot_examples += f"\n[样例输入]: {retrieved_input}\n[样例输出]: {retrieved_output}\n---"
        
        # d. 组合成最终的system prompt
        final_system_prompt = f"{base_system_prompt}\n\n{few_shot_examples}"
        
        full_prompt = (
            f"<|im_start|>system\n{final_system_prompt}<|im_end|>\n"
            f"<|im_start|>user\n{user_input}<|im_end|>\n"
            f"<|im_start|>assistant\n"
        )
        
        inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=128,
                do_sample=True,
                temperature=0.2,
                top_p=0.7,
                repetition_penalty=1.1,
                eos_token_id=tokenizer.eos_token_id
            )

        response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
        final_line = response.strip().split('\n')[-1] # 提取最后一行有效输出
        out_file.write(final_line + "\n")
        print(f"[{idx+1}/{len(test_data)}] 完成")

print("全部推理完成！")

正在加载检索工具...
加载LLM中...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

开始使用SRAG进行推理，结果将写入 output_srag_augmented.txt
[1/2000] 完成
[2/2000] 完成
[3/2000] 完成
[4/2000] 完成
[5/2000] 完成
[6/2000] 完成
[7/2000] 完成
[8/2000] 完成
[9/2000] 完成
[10/2000] 完成
[11/2000] 完成
[12/2000] 完成
[13/2000] 完成
[14/2000] 完成
[15/2000] 完成
[16/2000] 完成
[17/2000] 完成
[18/2000] 完成
[19/2000] 完成
[20/2000] 完成
[21/2000] 完成
[22/2000] 完成
[23/2000] 完成
[24/2000] 完成
[25/2000] 完成
[26/2000] 完成
[27/2000] 完成
[28/2000] 完成
[29/2000] 完成
[30/2000] 完成
[31/2000] 完成
[32/2000] 完成
[33/2000] 完成
[34/2000] 完成
[35/2000] 完成
[36/2000] 完成
[37/2000] 完成
[38/2000] 完成
[39/2000] 完成
[40/2000] 完成
[41/2000] 完成
[42/2000] 完成
[43/2000] 完成
[44/2000] 完成
[45/2000] 完成
[46/2000] 完成
[47/2000] 完成
[48/2000] 完成
[49/2000] 完成
[50/2000] 完成
[51/2000] 完成
[52/2000] 完成
[53/2000] 完成
[54/2000] 完成
[55/2000] 完成
[56/2000] 完成
[57/2000] 完成
[58/2000] 完成
[59/2000] 完成
[60/2000] 完成
[61/2000] 完成
[62/2000] 完成
[63/2000] 完成
[64/2000] 完成
[65/2000] 完成
[66/2000] 完成
[67/2000] 完成
[68/2000] 完成
[69/2000] 完成
[70/2000] 完成
[71/2000] 完成
[72/2000] 完成
[73/2000] 完成
[74/2000] 完成
[7

In [1]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

base_system_prompt = (
    "请从文本中抽取仇恨言论四元组，要求：\n"
    "1. 严格按照以下格式回复：(评论对象 | 论点 | 目标群体 | 是否仇恨 [END])，直接输出，不要解释。\n"
    "2. 如有多个四元组，两两之间用[SEP]分隔。\n"
    "3. 目标群体可以包含以下6项中的一项或多项：Region、Racism、Sexism、LGBTQ、others、non-hate。"
    "注意仅当‘是否仇恨’项为‘non-hate’时，‘目标群体’项才为‘non-hate’。"
)

retrieval_model = SentenceTransformer('./text2vec_local', device='cuda')
index = faiss.read_index('knowledge_base.index')
with open('knowledge_base_content.json', 'r', encoding='utf-8') as f:
    knowledge_base = json.load(f)

def debug_srag_case(user_input_text):
    """
    这个函数接收一个用户输入，执行SRAG流程，并打印出所有中间信息以供分析。
    """
    print("=============================================")
    print(f"🔍 正在诊断输入: '{user_input_text}'")
    print("=============================================\n")

    # --- 1. 检索 ---
    query_embedding = retrieval_model.encode([user_input_text], normalize_embeddings=True).astype('float32')
    k = 2
    distances, indices = index.search(query_embedding, k)

    print("--- 检索到的范例 ---")
    retrieved_examples_text = ""
    for i, idx in enumerate(indices[0]):
        retrieved_input = knowledge_base['user_inputs'][idx]
        retrieved_output = knowledge_base['assistant_responses'][idx]
        print(f"  范例 {i+1} (相似度: {distances[0][i]:.4f}):")
        print(f"    输入: '{retrieved_input}'")
        print(f"    输出: '{retrieved_output}'\n")
        retrieved_examples_text += f"[范例输入]: {retrieved_input}\n[范例输出]: {retrieved_output}\n---\n"

    # --- 2. 构建完整Prompt ---
    final_system_prompt = f"{base_system_prompt}\n\n下面是一些格式正确的范例：\n---\n{retrieved_examples_text}"
    full_prompt = (
        f"<|im_start|>system\n{final_system_prompt}<|im_end|>\n"
        f"<|im_start|>user\n{user_input_text}<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )
    
    print("--- 最终生成的完整Prompt ---")
    print(full_prompt)
    print("=============================================")


# --- 使用方法 ---
# 假设你已经加载了所有模型和知识库
# 从你的测试集中找一个得分变低的例子
failed_example_text = "有这样的话？如果是这样的，那就是卫辉官媒的错误。但是，你是不是在造谣。请给出网址以供查阅。"
debug_srag_case(failed_example_text)

🔍 正在诊断输入: '有这样的话？如果是这样的，那就是卫辉官媒的错误。但是，你是不是在造谣。请给出网址以供查阅。'

--- 检索到的范例 ---
  范例 1 (相似度: 0.7539):
    输入: '那你这不是造谣吗'
    输出: '你 | 造谣 | non-hate | non-hate [END]'

  范例 2 (相似度: 0.7190):
    输入: '而且网民根本不会过问你之前经历了多少，只会拿一句【不管怎么说也不能打人】来否认你之前承受过的所有折磨，【不能打人】这一句就是【无论如何都是你有罪】的最大筹码'
    输出: '不能打人 | 【无论如何都是你有罪】的最大筹码 | non-hate | non-hate [END]'

--- 最终生成的完整Prompt ---
<|im_start|>system
请从文本中抽取仇恨言论四元组，要求：
1. 严格按照以下格式回复：(评论对象 | 论点 | 目标群体 | 是否仇恨 [END])，直接输出，不要解释。
2. 如有多个四元组，两两之间用[SEP]分隔。
3. 目标群体可以包含以下6项中的一项或多项：Region、Racism、Sexism、LGBTQ、others、non-hate。注意仅当‘是否仇恨’项为‘non-hate’时，‘目标群体’项才为‘non-hate’。

下面是一些格式正确的范例：
---
[范例输入]: 那你这不是造谣吗
[范例输出]: 你 | 造谣 | non-hate | non-hate [END]
---
[范例输入]: 而且网民根本不会过问你之前经历了多少，只会拿一句【不管怎么说也不能打人】来否认你之前承受过的所有折磨，【不能打人】这一句就是【无论如何都是你有罪】的最大筹码
[范例输出]: 不能打人 | 【无论如何都是你有罪】的最大筹码 | non-hate | non-hate [END]
---
<|im_end|>
<|im_start|>user
有这样的话？如果是这样的，那就是卫辉官媒的错误。但是，你是不是在造谣。请给出网址以供查阅。<|im_end|>
<|im_start|>assistant

